# Importação das bibliotecas necessárias

In [1]:
import sys
import os
sys.path.append('/home/jovyan/work')

from utils.AlinharETL import AlinharETL

# Crie uma instância da classe AlinharETL

In [2]:
# Crie uma instância da classe
bucket = "bronze"
datamart = "FTPDadosAbertosReceita"
extrator_bronze = AlinharETL(bucket,datamart)

2024-09-25 18:59:37,252 - INFO - Iniciando Sessão Spark.


# Leitura da tabela

In [3]:
df_compliance_estabelecimentos  = extrator_bronze.criar_view_temporaria('bronze/DadosAbertosReceita/DadosAbertosReceitaEstabelecimentos','compliance_receita_estabelecimentos')

2024-09-25 18:59:40,610 - INFO - Aguarde... Criando View (bronze/DadosAbertosReceita/DadosAbertosReceitaEstabelecimentos)
2024-09-25 18:59:47,589 - INFO - View criada com sucesso


# Tratamentos na tabela 

In [4]:
sql_query = """
SELECT
    cnpj_basico AS dsc_cnpj_basico,  
    cnpj_ordem AS dsc_cnpj_ordem, 
    cnpj_dv AS dsc_cnpj_dv, 
    CASE 
        WHEN id_matriz_filial = 1 THEN 'Matriz' 
        ELSE 'Filial' 
    END AS dsc_matriz_filial, 
    nome_fantasia AS nm_fantasia, 
    CASE 
        WHEN situacao_cadastral = '01' THEN 'Nulo'
        WHEN situacao_cadastral = '02' THEN 'Ativa'
        WHEN situacao_cadastral = '03' THEN 'Suspensa'
        WHEN situacao_cadastral = '04' THEN 'Inapta'
        WHEN situacao_cadastral = '08' THEN 'Baixada'
        ELSE 'Outros' 
    END AS dsc_situacao_cadastral, 
    motivo_situacao_cadastral AS cd_motivo_situacao_cadastral,
    pais AS cd_pais,
    cnae_fiscal_principal AS cd_cnae_fiscal_principal, 
    cnae_fiscal_secundaria AS cd_cnae_fiscal_secundaria, 
    municipio AS cd_municipio,
    telefone_1 AS dsc_telefone_1,
    telefone_2 AS dsc_telefone_2
FROM compliance_receita_estabelecimentos
WHERE uf = 'ES'
"""
df_compliance_estabelecimentos  = extrator_bronze.carregar_dados_delta(sql_query)

2024-09-25 18:59:47,594 - INFO - Aguarde... Executando Query (Delta)
2024-09-25 18:59:47,595 - INFO - 
SELECT
    cnpj_basico AS dsc_cnpj_basico,  
    cnpj_ordem AS dsc_cnpj_ordem, 
    cnpj_dv AS dsc_cnpj_dv, 
    CASE 
        WHEN id_matriz_filial = 1 THEN 'Matriz' 
        ELSE 'Filial' 
    END AS dsc_matriz_filial, 
    nome_fantasia AS nm_fantasia, 
    CASE 
        WHEN situacao_cadastral = '01' THEN 'Nulo'
        WHEN situacao_cadastral = '02' THEN 'Ativa'
        WHEN situacao_cadastral = '03' THEN 'Suspensa'
        WHEN situacao_cadastral = '04' THEN 'Inapta'
        WHEN situacao_cadastral = '08' THEN 'Baixada'
        ELSE 'Outros' 
    END AS dsc_situacao_cadastral, 
    motivo_situacao_cadastral AS cd_motivo_situacao_cadastral,
    pais AS cd_pais,
    cnae_fiscal_principal AS cd_cnae_fiscal_principal, 
    cnae_fiscal_secundaria AS cd_cnae_fiscal_secundaria, 
    municipio AS cd_municipio,
    telefone_1 AS dsc_telefone_1,
    telefone_2 AS dsc_telefone_2
FROM compl

# Gravação no datalake

In [5]:
extrator_bronze.caminho_tabela_delta = 's3a://silver/DadosAbertosReceita/DadosAbertosReceitaEstabelecimentos'

In [6]:
extrator_bronze.salvar_delta(df_compliance_estabelecimentos, 'overwrite')

2024-09-25 18:59:47,745 - INFO - Aguarde... Persistindo dados (overwrite)
2024-09-25 19:00:09,453 - INFO - Dados persistidos com sucesso
2024-09-25 19:00:09,454 - INFO - s3a://silver/DadosAbertosReceita/DadosAbertosReceitaEstabelecimentos
2024-09-25 19:00:09,455 - INFO - Aguarde... Realizando optimize
2024-09-25 19:00:18,904 - INFO - Optimize executado com sucesso.
2024-09-25 19:00:18,904 - INFO - Aguarde... Realizando vacuum
2024-09-25 19:00:37,681 - INFO - Vacuum executado com sucesso.


# Encerra sessão spark

In [7]:
extrator_bronze.parar_sessao()

2024-09-25 19:00:37,971 - INFO - Sessão Spark finalizada.
